In [2]:
# The following code is a modified version of the code provided by Leap Motion, Inc. 
# This script is used to capture the hand data from the Leap Motion Controller and save it to a CSV file.
# The original code can be found at https://developer-archive.leapmotion.com/documentation/python/devguide/Leap_Hand.html

import pandas as pd
import numpy as np

'''
import sys
sys.path.append("../lib/Leap")
sys.path.append("../lib/")
sys.path.append("../lib/x86")
'''


import Leap, sys, thread, time
from Leap import CircleGesture, KeyTapGesture, ScreenTapGesture, SwipeGesture


In [3]:
class SampleListener(Leap.Listener):
    finger_names = ['Thumb', 'Index', 'Middle', 'Ring', 'Pinky']
    bone_names = ['Metacarpal', 'Proximal', 'Intermediate', 'Distal']
    state_names = ['STATE_INVALID', 'STATE_START', 'STATE_UPDATE', 'STATE_END']

    def on_init(self, controller):
        print("Initialized")
        
        controller.enable_gesture(Leap.Gesture.TYPE_CIRCLE)
        controller.enable_gesture(Leap.Gesture.TYPE_SWIPE)
        controller.enable_gesture(Leap.Gesture.TYPE_KEY_TAP)
        controller.enable_gesture(Leap.Gesture.TYPE_SCREEN_TAP)

        
        # Create DataFrame
        columns = ['Frame id', 'timestamp', 'hands', 'fingers', 'tools', 'gestures',
                  'handType', 'handID', 'handPalmPosition_X','handPalmPosition_Y','handPalmPosition_Z',
                  'pitch', 'roll', 'yaw','wristPosition_X', 'wristPosition_Y','wristPosition_Z',
                  'elbowPosition_X', 'elbowPosition_Y', 'elbowPosition_Z', 
                  'ToolID','ToolTipPosition','ToolDirection',
                  'GestureID','GestureType', 'GestureCircleClockwiseness', 'GestureCircleSweptAngle',
                  'GestureCircleProgress', 'GestureCircleRadius', 'GestureStateNames',
                  'SwipePosition','SwipeDirection','SwipeSpeed',
                  'KeytapPosition','KeytapDirection',
                  'ScreentapPosition','ScreentapDirection']
        
        finger_names = ['Thumb', 'Index', 'Middle', 'Ring', 'Pinky']
        bone_names = ['Metacarpal', 'Proximal', 'Intermediate', 'Distal']
            
        for finger in finger_names:
            columns.append(finger + 'ID')
            columns.append(finger + 'Length')
            columns.append(finger + 'Width')

        for finger in finger_names:
            for bone in bone_names:
                columns.append(finger + bone + 'Start_X')
                columns.append(finger + bone + 'Start_Y')
                columns.append(finger + bone + 'Start_Z')
                columns.append(finger + bone + 'End_X')
                columns.append(finger + bone + 'End_Y')
                columns.append(finger + bone + 'End_Z')
                columns.append(finger + bone + 'Direction_X') 
                columns.append(finger + bone + 'Direction_Y') 
                columns.append(finger + bone + 'Direction_Z')         
                
        
        
        self.df = pd.DataFrame(index=np.empty(0), columns=columns)      
        self.df = self.df.fillna(0)
        #index for DataFrame
        self.loc_i = 0
                

    def on_connect(self, controller):
        print("Connected")


    def on_disconnect(self, controller):
        # Note: not dispatched when running in a debugger.
        print("Disconnected")

    def on_exit(self, controller):
        #print(self.df)
        
        
        #compression_opts = dict(method='zip',archive_name='out.csv')  
        self.df.to_csv('data1.csv', index=False)#,compression=compression_opts) 

        print("Data was saved")
        print("Exited")

    def on_frame(self, controller):
        # Get the most recent frame and report some basic information
        frame = controller.frame()
       
        FEATURE_NUMBER = 226
        self.df.loc[self.loc_i] = [frame.id, frame.timestamp, len(frame.hands), len(frame.fingers),
                                len(frame.tools), len(frame.gestures()),
                                *(0 for _ in range(FEATURE_NUMBER))]
        
        print("Frame id: %d, timestamp: %d, hands: %d, fingers: %d, tools: %d, gestures: %d" % (
              frame.id, frame.timestamp, len(frame.hands), len(frame.fingers), len(frame.tools), len(frame.gestures())))


        # Get hands
        for hand in frame.hands:
            
            #'handType'            
            handType = "Left hand" if hand.is_left else "Right hand"
            self.df.iloc[self.loc_i, self.df.columns.get_loc('handType')] = handType

            
            #'handID', 'handPalmPosition'
            print("  %s, id %d, position: %s" % (
                handType, hand.id, hand.palm_position))            
            self.df.iloc[self.loc_i, self.df.columns.get_loc('handID')] = hand.id
            self.df.iloc[self.loc_i, self.df.columns.get_loc('handPalmPosition_X')] = hand.palm_position[0]
            self.df.iloc[self.loc_i, self.df.columns.get_loc('handPalmPosition_Y')] = hand.palm_position[1]
            self.df.iloc[self.loc_i, self.df.columns.get_loc('handPalmPosition_Z')] = hand.palm_position[2]

            
            # 'pitch', 'roll', 'yaw'
            # Get the hand's normal vector and direction
            normal = hand.palm_normal
            direction = hand.direction            
            # Calculate the hand's pitch, roll, and yaw angles
            print("  pitch: %f degrees, roll: %f degrees, yaw: %f degrees" % (
                direction.pitch * Leap.RAD_TO_DEG,
                normal.roll * Leap.RAD_TO_DEG,
                direction.yaw * Leap.RAD_TO_DEG))
            self.df.iloc[self.loc_i, self.df.columns.get_loc('pitch')] = direction.pitch * Leap.RAD_TO_DEG
            self.df.iloc[self.loc_i, self.df.columns.get_loc('roll')] = normal.roll * Leap.RAD_TO_DEG
            self.df.iloc[self.loc_i, self.df.columns.get_loc('yaw')] = direction.yaw * Leap.RAD_TO_DEG
            
            
            #'armDirection', 'wristPosition', 'elbowPosition'
            # Get arm bone
            arm = hand.arm
            print("  Arm direction: %s, wrist position: %s, elbow position: %s" % (
                arm.direction,
                arm.wrist_position,
                arm.elbow_position))            
            self.df.iloc[self.loc_i, self.df.columns.get_loc('wristPosition_X')] = arm.wrist_position[0]
            self.df.iloc[self.loc_i, self.df.columns.get_loc('wristPosition_Y')] = arm.wrist_position[1]
            self.df.iloc[self.loc_i, self.df.columns.get_loc('wristPosition_Z')] = arm.wrist_position[2]
            self.df.iloc[self.loc_i, self.df.columns.get_loc('elbowPosition_X')] = arm.elbow_position[0]
            self.df.iloc[self.loc_i, self.df.columns.get_loc('elbowPosition_Y')] = arm.elbow_position[1]
            self.df.iloc[self.loc_i, self.df.columns.get_loc('elbowPosition_Z')] = arm.elbow_position[2]
            
                        
            
            # Get fingers
            for finger in hand.fingers:
                print("   %s finger, id: %d, length: %fmm, width: %fmm" % (
                    self.finger_names[finger.type],
                    finger.id,
                    finger.length,
                    finger.width))
                #'ID', 'Length', 'Width' for each finger in hand.fingers              
                self.df.iloc[self.loc_i, self.df.columns.get_loc(self.finger_names[finger.type]+'ID')] = finger.id
                self.df.iloc[self.loc_i, self.df.columns.get_loc(self.finger_names[finger.type]+'Length')] = finger.length
                self.df.iloc[self.loc_i, self.df.columns.get_loc(self.finger_names[finger.type]+'Width')] = finger.width
                
                
                #'start', 'end', 'direction' for each bone for each finger
                # Get bones
                for b in range(0, 4):
                    bone = finger.bone(b)
                    print("      Bone: %s, start: %s, end: %s, direction: %s" % (
                        self.bone_names[bone.type],
                        bone.prev_joint,
                        bone.next_joint,
                        bone.direction))
                                                        
                    self.df.iloc[self.loc_i, self.df.columns.
                                 get_loc(self.finger_names[finger.type]+
                                         self.bone_names[bone.type]+'Start_X')] = bone.prev_joint[0]                                                          
                    self.df.iloc[self.loc_i, self.df.columns.
                                 get_loc(self.finger_names[finger.type]+
                                         self.bone_names[bone.type]+'Start_Y')] = bone.prev_joint[1]                                                          
                    self.df.iloc[self.loc_i, self.df.columns.
                                 get_loc(self.finger_names[finger.type]+
                                         self.bone_names[bone.type]+'Start_Z')] = bone.prev_joint[2]  
                    
                    
                    self.df.iloc[self.loc_i, self.df.columns.
                                 get_loc(self.finger_names[finger.type]+
                                         self.bone_names[bone.type]+'End_X')] = bone.next_joint[0]
                    self.df.iloc[self.loc_i, self.df.columns.
                                 get_loc(self.finger_names[finger.type]+
                                         self.bone_names[bone.type]+'End_Y')] = bone.next_joint[1]
                    self.df.iloc[self.loc_i, self.df.columns.
                                 get_loc(self.finger_names[finger.type]+
                                         self.bone_names[bone.type]+'End_Z')] = bone.next_joint[2]
                    
                    
                    
                    self.df.iloc[self.loc_i, self.df.columns.
                                 get_loc(self.finger_names[finger.type]+
                                         self.bone_names[bone.type]+'Direction_X')] = bone.direction[0]  
                    self.df.iloc[self.loc_i, self.df.columns.
                                 get_loc(self.finger_names[finger.type]+
                                         self.bone_names[bone.type]+'Direction_Y')] = bone.direction[1]
                    self.df.iloc[self.loc_i, self.df.columns.
                                 get_loc(self.finger_names[finger.type]+
                                         self.bone_names[bone.type]+'Direction_Z')] = bone.direction[2]
                    
                    
        # 'ToolID','ToolTipPosition','ToolDirection'
        # Get tools
        for tool in frame.tools:
            print("  Tool id: %d, position: %s, direction: %s" % (
                tool.id, tool.tip_position, tool.direction))
            self.df.iloc[self.loc_i, self.df.columns.get_loc('ToolID')] = tool.id
            self.df.iloc[self.loc_i, self.df.columns.get_loc('ToolTipPosition')] = tool.tip_position
            self.df.iloc[self.loc_i, self.df.columns.get_loc('ToolDirection')] = tool.direction

            

        # Get gestures
        for gesture in frame.gestures():
            #'GestureID','GestureType', 'GestureStateNames' 
            self.df.iloc[self.loc_i, self.df.columns.get_loc('GestureType')] = gesture.type
            self.df.iloc[self.loc_i, self.df.columns.get_loc('GestureID')] = gesture.id 
            self.df.iloc[self.loc_i, self.df.columns.get_loc('GestureStateNames')] = self.state_names[gesture.state]
            
            if gesture.type == Leap.Gesture.TYPE_CIRCLE:
                circle = CircleGesture(gesture) 
                # Determine clock direction using the angle between the pointable and the circle normal
                if circle.pointable.direction.angle_to(circle.normal) <= Leap.PI / 2:
                    clockwiseness = "clockwise"
                else:
                    clockwiseness = "counterclockwise"
                
                
                # Calculate the angle swept since the last frame
                swept_angle = 0
                if circle.state != Leap.Gesture.STATE_START:
                    previous_update = CircleGesture(controller.frame(1).gesture(circle.id))
                    swept_angle = (circle.progress - previous_update.progress) * 2 * Leap.PI
                    
                
                #'GestureCircleClockwiseness', 'GestureCircleSweptAngle','GestureCircleProgress','GestureCircleRadius'                   
                print("  Circle id: %d, %s, progress: %f, radius: %f, angle: %f degrees, %s" % (
                        gesture.id, self.state_names[gesture.state],
                        circle.progress, circle.radius, swept_angle * Leap.RAD_TO_DEG, clockwiseness))                
                self.df.iloc[self.loc_i, self.df.columns.get_loc('GestureCircleProgress')] = circle.progress
                self.df.iloc[self.loc_i, self.df.columns.get_loc('GestureCircleRadius')] = circle.radius 
                self.df.iloc[self.loc_i, self.df.columns.get_loc('GestureCircleSweptAngle')] = swept_angle * Leap.RAD_TO_DEG
                self.df.iloc[self.loc_i, self.df.columns.get_loc('GestureCircleClockwiseness')] = clockwiseness

                
            #'SwipePosition','SwipeDirection','SwipeSpeed'    
            if gesture.type == Leap.Gesture.TYPE_SWIPE:
                swipe = SwipeGesture(gesture)
                print("  Swipe id: %d, state: %s, position: %s, direction: %s, speed: %f" % (
                        gesture.id, self.state_names[gesture.state],
                        swipe.position, swipe.direction, swipe.speed))
                self.df.iloc[self.loc_i, self.df.columns.get_loc('SwipePosition')] = swipe.position
                self.df.iloc[self.loc_i, self.df.columns.get_loc('SwipeDirection')] = swipe.direction
                self.df.iloc[self.loc_i, self.df.columns.get_loc('SwipeSpeed')] = swipe.speed
                
            #'KeytapPosition','KeytapDirection'
            if gesture.type == Leap.Gesture.TYPE_KEY_TAP:
                keytap = KeyTapGesture(gesture)
                print("  Key Tap id: %d, %s, position: %s, direction: %s" % (
                        gesture.id, self.state_names[gesture.state],
                        keytap.position, keytap.direction))
                self.df.iloc[self.loc_i, self.df.columns.get_loc('KeytapPosition')] = keytap.position
                self.df.iloc[self.loc_i, self.df.columns.get_loc('KeytapDirection')] = keytap.direction
                
            #'ScreentapPosition','ScreentapDirection'
            if gesture.type == Leap.Gesture.TYPE_SCREEN_TAP:
                screentap = ScreenTapGesture(gesture)
                print("  Screen Tap id: %d, %s, position: %s, direction: %s" % (
                        gesture.id, self.state_names[gesture.state],
                        screentap.position, screentap.direction))
                self.df.iloc[self.loc_i, self.df.columns.get_loc('ScreentapPosition')] = screentap.position
                self.df.iloc[self.loc_i, self.df.columns.get_loc('ScreentapDirection')] = screentap.direction

        if not (frame.hands.is_empty and frame.gestures().is_empty):
            print("")
        
        self.loc_i += 1

    def state_string(self, state):
        if state == Leap.Gesture.STATE_START:
            return "STATE_START"

        if state == Leap.Gesture.STATE_UPDATE:
            return "STATE_UPDATE"

        if state == Leap.Gesture.STATE_STOP:
            return "STATE_STOP"

        if state == Leap.Gesture.STATE_INVALID:
            return "STATE_INVALID"

In [18]:
def main():
    # Create a sample listener and controller
    listener = SampleListener()
    controller = Leap.Controller()
    
    # Enable gestures
    controller.enable_gesture(Leap.Gesture.TYPE_CIRCLE)
    controller.enable_gesture(Leap.Gesture.TYPE_KEY_TAP)
    controller.enable_gesture(Leap.Gesture.TYPE_SCREEN_TAP)
    controller.enable_gesture(Leap.Gesture.TYPE_SWIPE)
    
    
    # Have the sample listener receive events from the controller
    controller.add_listener(listener)

    # Keep this procesыs running until Enter is pressed
    print("Press Enter to quit...")
    try:
        #sys.stdin.readline() # uncomment for console applications
        raw_input() # it an analogue of the upper line for Jupyter Notebook
    except KeyboardInterrupt:
        pass
    finally:
        # Remove the sample listener when done
        controller.remove_listener(listener)
if __name__ == "__main__":
    main()

Initialized
Connected
Press Enter to quit...
Frame id: 222688, timestamp: 41939383008, hands: 1, fingers: 5, tools: 0, gestures: 0
  Right hand, id 197, position: (-248.206, 130.227, 42.2682)
  pitch: 85.391907 degrees, roll: -13.244247 degrees, yaw: -88.971350 degrees
  Arm direction: (-0.750408, 0.195943, -0.631265), wrist position: (-181.388, 114.348, 41.9279), elbow position: (5.54675, 65.5365, 199.183)
   Thumb finger, id: 1970, length: 47.120438mm, width: 18.150467mm
      Bone: Metacarpal, start: (-194.542, 123.149, 60.5399), end: (-194.542, 123.149, 60.5399), direction: (0, 0, 0)
      Bone: Proximal, start: (-194.542, 123.149, 60.5399), end: (-226.906, 130.279, 92.073), direction: (0.707493, -0.155862, -0.689319)
      Bone: Intermediate, start: (-226.906, 130.279, 92.073), end: (-254.242, 134.297, 105.832), direction: (0.88563, -0.130195, -0.445767)
      Bone: Distal, start: (-254.242, 134.297, 105.832), end: (-270.433, 135.243, 104.762), direction: (0.996133, -0.05817, 0.06

      Bone: Distal, start: (73.1845, 178.54, -14.8472), end: (69.4805, 184.475, -26.837), direction: (0.266822, -0.427586, 0.863699)
   Ring finger, id: 2003, length: 60.798882mm, width: 16.202801mm
      Bone: Metacarpal, start: (64.8302, 104.372, 97.8618), end: (76.5407, 125.832, 45.493), direction: (-0.202624, -0.371313, 0.906129)
      Bone: Proximal, start: (76.5407, 125.832, 45.493), end: (80.2182, 140.954, 6.4537), direction: (-0.087503, -0.359824, 0.928908)
      Bone: Intermediate, start: (80.2182, 140.954, 6.4537), end: (78.0981, 151.289, -17.5033), direction: (0.0809894, -0.39478, 0.915199)
      Bone: Distal, start: (78.0981, 151.289, -17.5033), end: (75.4165, 156.898, -29.6084), direction: (0.197056, -0.412204, 0.889526)
   Pinky finger, id: 2004, length: 46.844017mm, width: 14.392593mm
      Bone: Metacarpal, start: (60.0471, 93.3554, 92.5461), end: (74.5038, 106.429, 42.724), direction: (-0.270223, -0.244376, 0.931268)
      Bone: Proximal, start: (74.5038, 106.429, 42.7

   Middle finger, id: 2002, length: 61.065281mm, width: 17.027557mm
      Bone: Metacarpal, start: (126.621, 92.3241, 100.595), end: (149.354, 115.386, 44.9636), direction: (-0.353171, -0.358271, 0.864241)
      Bone: Proximal, start: (149.354, 115.386, 44.9636), end: (163.621, 123.783, 4.07441), direction: (-0.323405, -0.19035, 0.926918)
      Bone: Intermediate, start: (163.621, 123.783, 4.07441), end: (165.912, 127.041, -21.5186), direction: (-0.088448, -0.125781, 0.988107)
      Bone: Distal, start: (165.912, 127.041, -21.5186), end: (165.663, 128.275, -34.5657), direction: (0.0190098, -0.0941709, 0.995375)
   Ring finger, id: 2003, length: 57.794380mm, width: 16.202801mm
      Bone: Metacarpal, start: (130.068, 82.4019, 96.7029), end: (154.134, 95.2466, 45.7521), direction: (-0.416409, -0.222251, 0.881594)
      Bone: Proximal, start: (154.134, 95.2466, 45.7521), end: (168.465, 101.812, 8.611), direction: (-0.355174, -0.162727, 0.920528)
      Bone: Intermediate, start: (168.465, 

      Bone: Proximal, start: (79.2242, 104.702, 84.2608), end: (85.9081, 144.398, 61.6994), direction: (-0.144843, -0.860223, 0.488913)
      Bone: Intermediate, start: (85.9081, 144.398, 61.6994), end: (91.5402, 163.368, 37.3102), direction: (-0.179324, -0.604001, 0.776547)
      Bone: Distal, start: (91.5402, 163.368, 37.3102), end: (94.5838, 172.137, 23.579), direction: (-0.183637, -0.529056, 0.828479)
   Index finger, id: 2001, length: 54.468266mm, width: 17.368898mm
      Bone: Metacarpal, start: (99.8827, 105.607, 87.6225), end: (118.706, 135.249, 29.5121), direction: (-0.277235, -0.436581, 0.855884)
      Bone: Proximal, start: (118.706, 135.249, 29.5121), end: (127.664, 144.273, -8.33312), direction: (-0.224375, -0.226041, 0.947925)
      Bone: Intermediate, start: (127.664, 144.273, -8.33312), end: (127.964, 147.804, -30.4301), direction: (-0.0134006, -0.157775, 0.987384)
      Bone: Distal, start: (127.964, 147.804, -30.4301), end: (125.571, 148.821, -42.2749), direction: (0.

   Ring finger, id: 2003, length: 57.448666mm, width: 16.232307mm
      Bone: Metacarpal, start: (100.271, 70.5147, 77.502), end: (121.601, 81.2608, 24.8577), direction: (-0.368963, -0.185892, 0.910665)
      Bone: Proximal, start: (121.601, 81.2608, 24.8577), end: (133.337, 87.3272, -13.0396), direction: (-0.292432, -0.151152, 0.944265)
      Bone: Intermediate, start: (133.337, 87.3272, -13.0396), end: (137.491, 89.3502, -37.2287), direction: (-0.168675, -0.082146, 0.982243)
      Bone: Distal, start: (137.491, 89.3502, -37.2287), end: (138.576, 89.8066, -49.9294), direction: (-0.0850749, -0.0357838, 0.995732)
   Pinky finger, id: 2004, length: 43.009830mm, width: 14.418804mm
      Bone: Metacarpal, start: (99.0318, 57.4589, 74.8678), end: (123.552, 61.3683, 27.4644), direction: (-0.458214, -0.0730551, 0.885835)
      Bone: Proximal, start: (123.552, 61.3683, 27.4644), end: (133.645, 65.761, -1.11047), direction: (-0.329588, -0.143452, 0.933163)
      Bone: Intermediate, start: (133.

...

      Bone: Intermediate, start: (180.136, 137.136, 2.98428), end: (183.178, 140.44, -19.1722), direction: (-0.134548, -0.14615, 0.98007)
      Bone: Distal, start: (183.178, 140.44, -19.1722), end: (181.513, 142.64, -31.1136), direction: (0.135865, -0.179502, 0.97433)
   Middle finger, id: 2012, length: 61.331097mm, width: 17.042048mm
      Bone: Metacarpal, start: (122.253, 108.855, 87.9793), end: (162.686, 112.859, 38.0314), direction: (-0.627973, -0.0621822, 0.775747)
      Bone: Proximal, start: (162.686, 112.859, 38.0314), end: (180.682, 114.022, -2.42281), direction: (-0.406315, -0.0262637, 0.913356)
      Bone: Intermediate, start: (180.682, 114.022, -2.42281), end: (181.708, 117.649, -28.1658), direction: (-0.0394491, -0.139429, 0.989446)
      Bone: Distal, start: (181.708, 117.649, -28.1658), end: (178.312, 120.48, -40.569), direction: (0.257917, -0.214964, 0.94195)
   Ring finger, id: 2013, length: 58.377754mm, width: 16.216589mm
      Bone: Metacarpal, start: (120.217, 98.

      Bone: Intermediate, start: (94.928, 116.338, -6.79559), end: (89.7484, 120.62, -22.7015), direction: (0.299963, -0.247992, 0.921153)
      Bone: Distal, start: (89.7484, 120.62, -22.7015), end: (83.7575, 124.84, -31.9998), direction: (0.506044, -0.35645, 0.785406)

Frame id: 225629, timestamp: 41966582600, hands: 1, fingers: 5, tools: 0, gestures: 0
  Right hand, id 201, position: (1.34013, 154.189, 51.1181)
  pitch: 19.401875 degrees, roll: -91.873128 degrees, yaw: 2.247368 degrees
  Arm direction: (-0.190123, 0.204439, -0.960239), wrist position: (4.86734, 125.491, 114.989), elbow position: (52.2412, 74.55, 354.256)
   Thumb finger, id: 2010, length: 48.213623mm, width: 18.165913mm
      Bone: Metacarpal, start: (-7.0289, 154.763, 107.157), end: (-7.0289, 154.763, 107.157), direction: (0, 0, 0)
      Bone: Proximal, start: (-7.0289, 154.763, 107.157), end: (-18.0743, 194.272, 85.8165), direction: (0.238858, -0.854389, 0.461482)
      Bone: Intermediate, start: (-18.0743, 194.27

      Bone: Intermediate, start: (166.166, 68.0134, 19.611), end: (168.895, 71.1782, 3.26632), direction: (-0.161739, -0.187592, 0.968839)
      Bone: Distal, start: (168.895, 71.1782, 3.26632), end: (165.362, 75.9349, -6.68533), direction: (0.305011, -0.410698, 0.859241)

Frame id: 225726, timestamp: 41967467732, hands: 1, fingers: 5, tools: 0, gestures: 0
  Right hand, id 201, position: (-74.836, 138.447, 87.363)
  pitch: 0.458676 degrees, roll: -61.671555 degrees, yaw: -56.492318 degrees
  Arm direction: (-0.584458, 0.111449, -0.803733), wrist position: (-15.701, 131.449, 123.749), elbow position: (129.931, 103.679, 324.019)
   Thumb finger, id: 2010, length: 47.830097mm, width: 18.165913mm
      Bone: Metacarpal, start: (-37.1684, 154.428, 125.959), end: (-37.1684, 154.428, 125.959), direction: (0, 0, 0)
      Bone: Proximal, start: (-37.1684, 154.428, 125.959), end: (-78.6223, 174.503, 124.838), direction: (0.89975, -0.435727, 0.0243353)
      Bone: Intermediate, start: (-78.6223,

      Bone: Intermediate, start: (22.2219, 159.435, -11.6437), end: (-0.339666, 163.981, -23.053), direction: (0.878302, -0.176955, 0.444154)
      Bone: Distal, start: (-0.339666, 163.981, -23.053), end: (-13.1562, 166.132, -22.2651), direction: (0.984398, -0.165221, -0.0605179)
   Ring finger, id: 2013, length: 57.555283mm, width: 16.216589mm
      Bone: Metacarpal, start: (56.8733, 132.922, 86.1786), end: (39.1379, 133.785, 31.1646), direction: (0.306796, -0.0149416, 0.951658)
      Bone: Proximal, start: (39.1379, 133.785, 31.1646), end: (21.9238, 133.833, -4.89223), direction: (0.430833, -0.0011835, 0.902431)
      Bone: Intermediate, start: (21.9238, 133.833, -4.89223), end: (-1.08265, 138.081, -12.9565), direction: (0.929696, -0.171662, 0.32588)
      Bone: Distal, start: (-1.08265, 138.081, -12.9565), end: (-13.4659, 140.976, -11.2486), direction: (0.965076, -0.225636, -0.133103)
   Pinky finger, id: 2014, length: 44.729469mm, width: 14.404841mm
      Bone: Metacarpal, start: (

Data was saved
Exited
